In [1]:
import requests as rq
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as soup
import re
import time
import csv
import pickle
from pprint import pprint

In [2]:
#Proxies need to be changed each run until data is collected!
proxies = {
    "http": 'http://59.65.35.102:3128', 
    "https": 'http://59.65.35.102:3128'
}
prox2 = {
    "http": 'http://45.32.193.119:8118', 
    "https": 'http://45.32.193.119:8118'
}
prox3 = {
    "http": 'http://66.215.16.23:8080', 
    "https": 'http://66.215.16.23:8080'
}
prox4 = {
    "http": 'http://159.65.47.178:8888', 
    "https": 'http://159.65.47.178:8888'
}
superbase = 'https://steepster.com/'
start_url = 'https://steepster.com/teas?green=on'
search_base = 'https://steepster.com/teas?green=on&page=' #Add page number to end
ttype_list = ['Green Tea','Black Tea','Pu-Erh Tea','Oolong Tea','White Tea','Matcha Tea','Yellow Tea']

In [3]:
response = rq.get('https://genius.com/Bob-the-builder-can-we-fix-it-lyrics',prox4)

In [4]:
page = response.text
mainpage = soup(page,"lxml")

In [5]:
text = mainpage.find_all(class_='referent')
str(text[0]).split('>')[1].replace('</a','')

'Bob the Builder!'

In [22]:
#Getting all cases where we have a /talks url
list_1 = mainpage.find_all(class_='entry-count')

In [249]:
def getTeas(color):
    review_url = []
    descpage = []
    proxy=None
    base = 'https://steepster.com/teas?'
    base = base+color
    for j in range(6):
        response = rq.get(base+str(j+1),proxy)
        page = response.text
        mainpage = soup(page,"lxml")
        #Getting all cases where we have a /tea url
        list_1 = mainpage.find_all(class_='entry-count')
        time.sleep(.5)
        for i in list_1:
            ret = re.findall("/teas/[a-zA-Z\-/0-9]+",str(i))
            descpage.append(superbase+ret[0])
            time.sleep(.5)
        if j ==3:
            proxy=proxies
    return descpage

In [163]:
len(descpage)

168

In [74]:
response = rq.get('https://steepster.com/teas/bird-pick-tea-and-herb/5862-premium-silky-green-tea')

In [75]:
page = response.text
reviewp = soup(page,"lxml")

In [238]:
def getReviews(reviewpage, currenttea, proxy):
    reviewstr=[]
    review_list=[]
    if str(reviewpage.find_all('div', class_='text')):
        kj = reviewpage.find_all('a')[-14:-12]
        reviewstr = str(reviewpage.find_all('div', class_='text')).replace('</p>\n</span>\n</div>, <div class="text">\n','')\
                .replace('\n<p>','').replace('<br/>\n',' ').replace('<span class="caps">','')\
                .replace('</span>','').replace('</p>','').replace(']','').replace('\n\n</div>','').split('<span itemprop="description">')
        review_list += reviewstr[1:]      
        if kj[0].get('href') == '?page=2#tasting-notes':
            next_1 = kj[0].get('href')
            response = rq.get(currenttea+next_1, proxy)
            page = response.text
            reviewp2 = soup(page,"lxml")
            reviewstr = str(reviewp2.find_all('div', class_='text')).replace('</p>\n</span>\n</div>, <div class="text">\n','')\
                .replace('\n<p>','').replace('<br/>\n',' ').replace('<span class="caps">','')\
                .replace('</span>','').replace('</p>','').replace(']','').replace('\n\n</div>','').split('<span itemprop="description">')
            review_list += reviewstr[1:] 
        time.sleep(.5)
        if kj[1].get('href') == '?page=3#tasting-notes':
            next_ = kj[1].get('href')
            response = rq.get(currenttea+next_, proxy)
            page = response.text
            reviewp3 = soup(page,"lxml")
            reviewstr = str(reviewp3.find_all('div', class_='text')).replace('</p>\n</span>\n</div>, <div class="text">\n','')\
                .replace('\n<p>','').replace('<br/>\n',' ').replace('<span class="caps">','')\
                .replace('</span>','').replace('</p>','').replace(']','').replace('\n\n</div>','').split('<span itemprop="description">')
            review_list += reviewstr[1:]  
        time.sleep(.5)
    return review_list


In [239]:
def updateDict(url_list, old_dict, teatype):
    dict_list =[]
    count=0
    proxy = proxies
    for i in url_list:
        time.sleep(.5)
        new_dict = {}
        teaname = ''
        teaing = ''
        teaflav = ''
        tot_reviews = []
        response = rq.get(i, proxy)
        page = response.text
        reviewp = soup(page,"lxml")
        teaname = str(reviewp.h1).split('"')[3]
        teaing = str(reviewp.find_all('dd')[1]).split('>')[1].replace('</dd','')
        teaflav = str(reviewp.find_all('dd')[2]).replace('<dd>', '').replace('</dd>', '')
        tot_reviews = getReviews(reviewp,i,proxy)
        new_dict['Tea Name'] =teaname
        new_dict['Tea Ingredients'] =teaing
        new_dict['Tea Flavors'] =teaflav
        new_dict['Tea Reviews'] =tot_reviews
        dict_list.append(new_dict)
        count+=1
        if count==8:
            proxy=prox2
        elif count==16:
            proxy=prox3
        elif count==24:
            proxy=prox4
        elif count==32:
            proxy=proxies
            count=0
    old_dict[teatype]=dict_list
    return old_dict

In [252]:
comurl = ['green=on&page=','black=on&page=','pu-erh=on&page=','oolong=on&page=','white=on&page=','matcha=on&page=','yellow=on&page=']
tot_dict = {}
for index, i in enumerate(comurl):
    descpage = getTeas(i)
    tot_dict = updateDict(descpage, tot_dict, ttype_list[index])

In [254]:
descpage

['https://steepster.com//teas/rishi-tea/5683-ancient-yellow-buds',
 'https://steepster.com//teas/teavana/43360-golden-dragon-yellow-tea',
 'https://steepster.com//teas/mark-t-wendell/23148-huo-shan-yellow-sprouting',
 'https://steepster.com//teas/the-jasmine-pearl-tea-merchants/40313-yellow-mu-dan',
 'https://steepster.com//teas/davidstea/52871-yellow-gem',
 'https://steepster.com//teas/ten-ren/30347-sun-moon-lake-high-mountain-oolong-tea',
 'https://steepster.com//teas/mandala-tea/27557-yellow-buds-huo-shan-huang-ya',
 'https://steepster.com//teas/august-uncommon-tea/56208-the-settlement',
 'https://steepster.com//teas/what-cha/48466-huoshan-huang-ya-yellow',
 'https://steepster.com//teas/harney-and-sons/21228-hunan-yellow-sprout-2012',
 'https://steepster.com//teas/life-in-teacup/11912-meng-ding-snow-bud-xue-ya',
 'https://steepster.com//teas/tribute-tea-company/42429-huo-mountain-yellow-buds-yellow-tea',
 'https://steepster.com//teas/canton-tea-co/2985-meng-ding-huang-ya',
 'https:/

In [256]:
for i in tot_dict:
    print(len(tot_dict[i]))

168
168
168
168
168
168
168


In [257]:
#Writing a pickle file to save data

with open('tea_data.pkl', 'wb') as picklefile:
    pickle.dump(tot_dict, picklefile)

In [136]:
if str(reviewp.find_all('div', class_='text')):
    reviewstr = str(reviewp.find_all('div', class_='text')).replace('</p>\n</span>\n</div>, <div class="text">\n','')\
                .replace('\n<p>','').replace('<br/>\n',' ').replace('<span class="caps">','')\
                .replace('</span>','').replace('</p>','').replace(']','').replace('\n\n</div>','').split('<span itemprop="description">')
reviewstr = reviewstr[1:] 

In [149]:
turnpg = []
specurl = currentea[0]
kj = reviewp.find_all('a')[-14:-12]
if kf[0] == '?page=2#tasting-notes':
    next_ = kf[0].get('href')
    turnpg.append(specurl+next_)
    turnpg
if kf[1] == '?page=3#tasting-notes':
    next_ = kf[0].get('href')
    turnpg.append(specurl+next_)
    turnpg

['?page=2#tasting-notes', '?page=3#tasting-notes']